In [1]:
import pandas as pd

In [2]:
database=pd.read_csv('./FinalDataBase.csv')

In [3]:
database

,Unnamed: 0,Stock,Start Date,End Date,Accord Code,Sr.No.,Company Name,CD_BSE Code,CD_Bse Scrip Name,CD_NSE Symbol,CD_ISIN No
0,0,ABB,29-12-2006,01-02-2013,100002,33,ABB India Ltd.,500002.0,ABB LTD.,ABB,INE117A01022
1,1,SUZLON,29-12-2006,01-03-2013,132667,3504,Suzlon Energy Ltd.,532667.0,SUZLONENERGY,SUZLON,INE040H01021
2,2,GUJFLUORO,05-08-2011,01-03-2013,295872,1278,Gujarat Fluorochemicals Ltd.,542812.0,FLUOROCHEM,FLUOROCHEM,INE09N301011
3,3,GVKPIL,21-08-2008,01-04-2013,132708,1298,GVK Power & Infrastructure Ltd.,532708.0,GVK POWERINF,GVKPIL,INE251H01024
4,4,BHUSANSTL,05-09-2007,01-04-2013,100055,3571,Tata Steel BSL Ltd.,500055.0,TATASTLBSL,TATASTLBSL,INE824B01021
...,...,...,...,...,...,...,...,...,...,...,...
314,314,LT,29-12-2006,31-12-2024,100510,1944,Larsen & Toubro Ltd.,500510.0,LARSEN & TOU,LT,INE018A01030
315,315,LUPIN,29-12-2006,31-12-2024,100257,2002,Lupin Ltd.,500257.0,LUPIN LTD.,LUPIN,INE326A01037
316,316,M_M,29-12-2006,31-12-2024,100520,2041,Mahindra & Mahindra Ltd.,500520.0,MAH & MAH,M&M,INE101A01026
317,317,RBLBANK,26-05-2017,31-12-2024,209068,2949,RBL Bank Ltd.,540065.0,RBLBANK,RBLBANK,INE976G01028


In [4]:
stockuniverse=pd.read_csv('./StockUniverseWithFutPrice.csv')
stockuniverse['End'] = pd.to_datetime(stockuniverse['End'])
stockuniverse['Month'] = stockuniverse['End'].dt.month
stockuniverse['Year'] = stockuniverse['End'].dt.year
stockuniverse
year_month_map = stockuniverse[['Year', 'Month']].drop_duplicates().reset_index(drop=True)

In [5]:
import numpy as np

def calculate_cagr(initial_value, final_value, num_years):
    if initial_value <= 0 or final_value <= 0 or num_years <= 0:
        return 0
    cagr = (final_value / initial_value) ** (1 / num_years) - 1
    return round(cagr * 100, 2)

# Initialize variables
exposure = 30
map = None  # Initialize as None
removed_exposure = 0  # Variable to store removed values
prev_expo = exposure
global_val = []
start_year = 2013
end_year = 2022

# Track initial and final portfolio values for CAGR calculation
initial_portfolio_value = exposure
final_portfolio_value = None

for _, row in year_month_map.iterrows():
        year= row['Year']
        month = row['Month']
        stocks = stockuniverse[(stockuniverse['Month'] == month) & (stockuniverse['Year'] == year)]
        
        if map is None:  # Initialize map if it's None
            map = {stock['Symbol']: exposure / len(stocks) for _, stock in stocks.iterrows()}

            for key, value in map.items():
                obj = {
                    'Date': f"{year}_{month} ",
                    "Symbol": key,
                    "Value": value,
                    'type': "Start"
                }
                global_val.append(obj)

        # Find overlap and differences
        stock_symbols = set(stocks['Symbol'].to_list())  # Convert to set for fast operations
        map_keys = set(map.keys())
      
        common_symbols = stock_symbols.intersection(map_keys)
        stocks_only = stock_symbols - map_keys
        map_only = map_keys - stock_symbols

        # Collect and remove symbols in `map` but not in `stocks`
        for symbol in map_only:
            removed_exposure += map.pop(symbol)
  
        if stocks_only:
            # Divide the removed exposure equally among the new stocks
            exposure_per_stock = removed_exposure / len(stocks_only)
            for symbol in stocks_only:
                map[symbol] = exposure_per_stock

        total_exposure = sum(map.values())
        removed_exposure = 0

        # Process stocks
        for _, stock in stocks.iterrows():
            percentage = (stock['Curr Price'] - stock['Prev Price']) / stock['Prev Price']
            if stock['Symbol'] in map:
                map[stock['Symbol']] *= (1 + percentage)
            
        for key, value in map.items():
            obj = {
                'Date': f"{year}_{month} ",
                "Symbol": key,
                "Value": value,
                'type': "Close"
            }
            global_val.append(obj)

        total_exposure = sum(map.values())
        percentage_Returns = round(((total_exposure - prev_expo) / prev_expo) * 100, 2)
        prev_expo = total_exposure

        # Track final portfolio value in the last iteration
        if year == end_year and month == 12:
            final_portfolio_value = total_exposure

final_exposure = sum(map.values())

# Calculate CAGR
cagr = calculate_cagr(initial_portfolio_value, final_portfolio_value, end_year - start_year + 1)

print("Initial Portfolio Value:", initial_portfolio_value)
print("Final Portfolio Value:", final_portfolio_value)
print("CAGR:", cagr, "%")
print("Global Values:", global_val)
print("Final Exposure:", final_exposure)

Initial Portfolio Value: 30
Final Portfolio Value: 39.84368194523067
CAGR: 2.88 %
Global Values: [{'Date': '2013_1 ', 'Symbol': 'UNITDSPR', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'KTKBANK', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'DENABANK', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'AUROPHARMA', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'MRF', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'OFSS', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'ASIANPAINT', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'GODREJIND', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'ULTRACEMCO', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'COLPAL', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'HDFCBANK', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Symbol': 'PTC', 'Value': 1.0, 'type': 'Start'}, {'Date': '2013_1 ', 'Sym

In [6]:
import numpy as np
import math

def calculate_cagr(initial_value, final_value, num_years):
    if initial_value <= 0 or final_value <= 0 or num_years <= 0:
        return 0
    cagr = (final_value / initial_value) ** (1 / num_years) - 1
    return round(cagr * 100, 2)

# Initialize variables
exposure = 10000000
map = None  # Initialize as None
removed_exposure = 0  # Variable to store removed values
prev_expo = exposure
global_val = []
start_year = 2013
end_year = 2022
map_qt={}

# Track initial and final portfolio values for CAGR calculation
initial_portfolio_value = exposure
final_portfolio_value = None
balance=0
months=[1,7]
for _, row in year_month_map.iterrows():
        year= row['Year']
        month = row['Month']
        stocks = stockuniverse[(stockuniverse['Month'] == month) & (stockuniverse['Year'] == year)]
        if map is None: 
            map = {}
            map_qt = {stock['Symbol']: math.floor((exposure / len(stocks)) / stock['Prev Price']) for _, stock in stocks.iterrows()}
            for _, stock in stocks.iterrows():
                map[stock['Symbol']] = math.floor(map_qt[stock['Symbol']] * stock['Prev Price'])
                balance+=(exposure / len(stocks))-math.floor(map_qt[stock['Symbol']] * stock['Prev Price'])      

        stock_symbols = set(stocks['Symbol'].to_list()) 
        map_keys = set(map.keys())
        common_symbols = stock_symbols.intersection(map_keys)
        stocks_only = stock_symbols - map_keys
        map_only = map_keys - stock_symbols
        
        for symbol in map_only:
            removed_exposure += map.pop(symbol)
            map_qt.pop(symbol)
 

        if stocks_only:
            exposure_per_stock = (removed_exposure+balance) / len(stocks_only)
            balance=0
            for symbol in stocks_only:
                stock_details=stocks[stocks['Symbol']==symbol].iloc[0]
                map_qt[symbol] = math.floor(exposure_per_stock/ stock_details['Prev Price'])
                map[symbol] = math.floor(map_qt[symbol] * stock_details['Prev Price'])
                val=(exposure_per_stock-math.floor(map_qt[symbol] * stock_details['Prev Price']))
                balance+=val
   

        #map gets Corrected Here
        for _, stock in stocks.iterrows():
            want=map_qt[stock['Symbol']]*stock['Prev Price']   
            map[stock['Symbol']]=want

        prev_map=map.copy()

        total_exposure = sum(map.values())
        removed_exposure = 0

        # Process stocks
        for _, stock in stocks.iterrows():
            percentage = (stock['Curr Price'] - stock['Prev Price']) / stock['Prev Price']
            if stock['Symbol'] in map:
                map[stock['Symbol']] *= (1 + percentage)


            
        for key, value in map.items():
            pm=prev_map[key]
            obj = {
                'Date': f"{year}_{month} ",
                'Start Value':pm,
                'Stock Start Price':stocks[stocks['Symbol']==key].iloc[0]['Prev Price'],
                'Stock Close Price':stocks[stocks['Symbol']==key].iloc[0]['Curr Price'],
                "Close Symbol": key,
                "Close Value": value,
                'Close Qty': map_qt[key],
                'Balance':balance
            }
            global_val.append(obj)

        total_exposure = sum(map.values())
        percentage_Returns = round(((total_exposure - prev_expo) / prev_expo) * 100, 2)
        prev_expo = total_exposure


        if year == end_year and month == 12:
            final_portfolio_value = total_exposure

final_exposure = sum(map.values())+balance
balance=0

# Calculate CAGR
cagr = calculate_cagr(initial_portfolio_value, final_portfolio_value, end_year - start_year + 1)

print("Initial Portfolio Value:", initial_portfolio_value)
print("Final Portfolio Value:", final_portfolio_value)
print("CAGR:", cagr, "%")
print("Global Values:", global_val)
print("Final Exposure:", final_exposure)
print("balance is:",balance)

Initial Portfolio Value: 10000000
Final Portfolio Value: 35730029.89999999
CAGR: 13.58 %
Global Values: [{'Date': '2013_1 ', 'Start Value': 333005.67, 'Stock Start Price': 379.71, 'Stock Close Price': 360.88, 'Close Symbol': 'UNITDSPR', 'Close Value': 316491.76, 'Close Qty': 877, 'Balance': 9679.999999999418}, {'Date': '2013_1 ', 'Start Value': 333222.5555939636, 'Stock Start Price': 124.105234858087, 'Stock Close Price': 121.994853477743, 'Close Symbol': 'KTKBANK', 'Close Value': 327556.18158773996, 'Close Qty': 2685, 'Balance': 9679.999999999418}, {'Date': '2013_1 ', 'Start Value': 333324.9, 'Stock Start Price': 114.9, 'Stock Close Price': 111.05, 'Close Symbol': 'DENABANK', 'Close Value': 322156.05, 'Close Qty': 2901, 'Balance': 9679.999999999418}, {'Date': '2013_1 ', 'Start Value': 333311.4, 'Stock Start Price': 95.45, 'Stock Close Price': 93.7, 'Close Symbol': 'AUROPHARMA', 'Close Value': 327200.4, 'Close Qty': 3492, 'Balance': 9679.999999999418}, {'Date': '2013_1 ', 'Start Value'

In [7]:
stockuniverse

,Unnamed: 0.1,Unnamed: 0,Symbol,Start,End,Company Name,Accord Code,Prev Date,Curr Price,Curr Date,Prev Price,Returns,Diff,Month,Year
0,0,0,UNITDSPR,2012-12-28,2013-01-31,United Spirits Ltd.,132432,2012-12-28,360.880000,2013-01-31,379.710000,-0.049590,34 days,1,2013
1,1,1,KTKBANK,2012-12-28,2013-01-31,The Karnataka Bank Ltd.,132652,2012-12-28,121.994853,2013-01-31,124.105235,-0.017005,34 days,1,2013
2,2,2,DENABANK,2012-12-28,2013-01-31,Dena Bank - (Amalgamated),132121,2012-12-28,111.050000,2013-01-31,114.900000,-0.033507,34 days,1,2013
3,3,3,AUROPHARMA,2012-12-28,2013-01-31,Aurobindo Pharma Ltd.,124804,2012-12-28,93.700000,2013-01-31,95.450000,-0.018334,34 days,1,2013
4,4,4,MRF,2012-12-28,2013-01-31,MRF Ltd.,100290,2012-12-28,13138.850000,2013-01-31,12694.750000,0.034983,34 days,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,3595,3595,ESCORTS,2022-11-25,2022-12-29,Escorts Kubota Ltd.,100495,2022-11-25,2158.550000,2022-12-29,2282.950000,-0.054491,34 days,12,2022
3596,3596,3596,CUMMINSIND,2022-11-25,2022-12-29,Cummins India Ltd.,100480,2022-11-25,1388.400000,2022-12-29,1389.950000,-0.001115,34 days,12,2022
3597,3597,3597,UNITDSPR,2022-11-25,2022-12-29,United Spirits Ltd.,132432,2022-11-25,878.150000,2022-12-29,889.350000,-0.012593,34 days,12,2022
3598,3598,3598,TATAMOTORS,2022-11-25,2022-12-29,Tata Motors Ltd.,100570,2022-11-25,386.150000,2022-12-29,437.500000,-0.117371,34 days,12,2022


In [8]:
df=pd.DataFrame(global_val)


In [9]:
df.to_csv('FutTradeSheet.csv')

In [10]:
df[df['Close Symbol']=='KTKBANK']

,Date,Start Value,Stock Start Price,Stock Close Price,Close Symbol,Close Value,Close Qty,Balance
1,2013_1,333222.555594,124.105235,121.994853,KTKBANK,327556.181588,2685,9680.0
31,2013_2,323082.728425,120.328763,103.593809,KTKBANK,278149.376656,2685,9680.0
61,2013_3,270594.211315,100.779967,96.818374,KTKBANK,259957.333794,2685,9680.0
91,2013_4,259957.333794,96.818374,112.664746,KTKBANK,302504.843876,2685,9680.0
121,2013_5,293458.527480,109.295541,105.445021,KTKBANK,283119.880171,2685,9680.0
151,2013_6,272184.772439,101.372355,80.416638,KTKBANK,215918.672658,2685,9680.0
